In [1]:
import os, re, json , glob

In [7]:
# import re
# from bs4 import BeautifulSoup

# # Sample README content (you can load this from a file instead)
# # readme_content = '''
# # <details>
# # <summary>Click the arrow to view options.</summary>

# #   - m: metre (unit for length).
# #   - Pa: pascal (unit for pressure).
# #   - K: kelvin jslj.

# # </details>
# # '''

def extract_bullets_with_brackets(html_text):
    soup = BeautifulSoup(html_text, "html.parser")
    results = {}

    bullet_pattern = re.compile(r"\s*-\s*(\w+):\s*([^\(]+?)(?:\s*\((.*?)\))?\.")

    for details in soup.find_all("details"):
        lines = details.get_text().splitlines()
        for line in lines:
            match = bullet_pattern.match(line)
            if match:
                symbol, description, bracket_info = match.groups()
                results[symbol] = {
                    "text1": description.strip(),
                    "text2": bracket_info.strip() if bracket_info else None
                }

    return results

# # # Run the function
# # bullets = extract_bullets_with_brackets(readme_content)

# # # Print neatly
# # from pprint import pprint
# # pprint(bullets)


In [8]:
rme = glob.glob('*/*README*')
rme

['native-horizontal-grid-type/README.md',
 'native-horizontal-grid-region/README.md',
 'model-calendar/README.md',
 'native-horizontal-grid-nominal-resolution/README.md',
 'model-component/README.md',
 'native-horizontal-grid-temporal-refinement/README.md',
 'native-horizontal-grid-grid-mapping/README.md',
 'native-horizontal-grid-truncation-method/README.md',
 'model-component-type/README.md',
 'native-vertical-grid-coordinate/README.md',
 'native-horizontal-grid-arrangement/README.md',
 'model/README.md',
 'reference/README.md',
 'native-vertical-grid-units/README.md',
 'grid/README.md']

In [57]:
# pm = list(DATA_DESCRIPTOR_CLASS_MAPPING.values())[-4]


def bullet_pydantic(pm):

    keys = ""

    for key, value in pm.__pydantic_fields__.items():
        rq = '✓' if value.is_required() else '✗'
        
        typename = getattr(value.annotation, '__name__', str(value.annotation))
        description = value.description or '<< No description in model >>'

        # print(f"- **`{key}`**  [**{typename}**]  {rq} : \n  {description}\n")
        # print(f"- `{key}` [{typename}] {rq} : {description}")
        # print(f"- `{key}`\n      Type       : [{typename}]\n      Required   : {rq}\n      Description: {description}\n")
        
        # print(f"- **`{key}`**  \n  {rq} (**{typename}**)\n  {description.rstrip()}\n  ")
        
        keys += f"- **`{key}`** (**{typename}**) \n  {description.rstrip()}\n"
        
        # keys += f"- **`{key}`**  \n  {rq} (**{typename}**)\n  {description.rstrip()}\n  "
        
        
    return keys

def bullet_names(keynames):
    keys = ""
    for key in keys:
        # print(f"- `{key}`")
        print(f"- **`{key}`**")
        keys += f"- **`{key}`**  \n  ? (**NoType**)\n  No Linked Pydantic Model \n  "
    return keys

In [45]:
# for rd in rme:
#     path = os.path.dirname(rd.replace('README.md',''))
#     print(path)
#     mytype = path.split('/')[-1]
    
#     context = {
#     "@context": [
#         "../_context_",
#         {
#             "@base": f"https://wcrp-cmip.github.io/WCRP-universe/{mytype}/",
#             "@vocab": f"https://wcrp-cmip.github.io/WCRP-universe/{mytype}/"
#         }
#     ],
#     "@embed": "@always",
#     "@type": [
#         "universal",
#         f"wcrp:{mytype}",
#     ]
#     }
#     with open(f'./{path}/_context_', 'w') as f:
#         json.dump(context, f, indent=4)
    
#     branches = extract_bullets_with_brackets(open(rd).read())
#     # print(branches)
    
#     for k, v in branches.items():
#         validation = k.replace('_','-').replace(' ','-')
#         filename = validation.lower()
        
        
#         ld = {
#             "id": filename,
#             "type": [f"wcrp:{mytype}","universal"],
#             "validation-key": validation,

            
#         } 
        
#         if v['text2'] is not None:
            
#             ld['description'] =  v['text2']
#             ld['ui-label'] = v['text1']
            
#         else:
#             ld['description'] = v['text1']
#             ld['ui-label'] = validation.replace('-',' ').capitalize()
            
#         # print(ld)
        
#         # with open(f'./{path}/{filename}.json', 'w') as f:
#         #     json.dump(ld, f, indent=4)

In [46]:
folders = glob.glob('*/')
folders

['native-horizontal-grid-type/',
 'source-type/',
 'native-horizontal-grid-region/',
 'empty/',
 'model-calendar/',
 'license/',
 'frequency/',
 'activity/',
 'native-horizontal-grid-temporal-refinement/',
 'product/',
 'mip/',
 'grid-label/',
 'model-component-type/',
 'resolution/',
 'native-vertical-grid-coordinate/',
 'organisation/',
 'realm/',
 'native-vertical-grid-units/',
 'model-family/']

In [ ]:
import numpy as np
import esgvoc
from esgvoc.api.data_descriptors import DATA_DESCRIPTOR_CLASS_MAPPING

from cmipld.utils.git import get_path_url,get_repo_url,get_relative_path,url2io
from cmipld import prefix_url,resolve_prefix

missing_pydantic = []

for dir in folders:
    
    print(dir)
    name = dir.strip('/')
    
    has_json = any(f.endswith('.json') for f in os.listdir(dir))
    if not has_json:
        print(f"Skipping {dir} as it does not contain any JSON files.")
        os.system(f'mv {dir} empty')
        continue
    
    # if content is empty, then delete folder. 
    
    try:
        keys = os.popen(f"jq -r 'recurse(.[]? // empty) | objects | keys_unsorted[]' {dir}*.json | sort | uniq -c | sort -nr").read()
        keynames = [i.split(' ')[-1] for i in keys.split('\n') if i.strip() != '']
        
        keynumbers = [int(i.split(' ')[-2]) for i in keys.split('\n') if i.strip() != '']
        avg = int(np.median(keynumbers))
        # print(keynames,keynumbers)
        
        
        
        different = [keynames[i] for i, x in enumerate(keynumbers) if x != avg]
        
        if different: print('The following keys are not present in all files:',different)


    except ValueError as e:
        # print(f"Error processing {dir}: {e}")
        continue

    
    dname = dir.strip('/').replace('-','_')
    pydantic = False
    
    match True:
        case _ if dname in DATA_DESCRIPTOR_CLASS_MAPPING:
            pydantic = dname  
        case _ if dname + '_new' in DATA_DESCRIPTOR_CLASS_MAPPING:
            pydantic = dname + '_new' 
        case _ if dir.strip('/') in DATA_DESCRIPTOR_CLASS_MAPPING:
            pydantic = dir.strip('/')
        case _:
            missing_pydantic.append(dname)  # none matched
            
            print(f"------ \n Adding {dname} to DATA_DESCRIPTOR_CLASS_MAPPING")
    
    
    
    if pydantic:
        bullets = bullet_pydantic(DATA_DESCRIPTOR_CLASS_MAPPING[pydantic])
    else:
        bulltes = bullet_names(keynames)
    

    ldurl= ''
    content = get_path_url(dir).replace('wolfiex','wcrp-cmip')
    repo = get_repo_url().replace('wolfiex','wcrp-cmip')
    relpath = get_relative_path(dir)
    
    
    
    io = relpath.replace('src-data/',url2io(repo,'main',relpath))
    short = prefix_url(io)
    
    
    # print(content,repo)
    # print(relpath,short,io),
    # break
    
    
    info = f'''

<section id="info">

# {name.title().replace('-',' ').replace(':',' : ')}  ({short.split(':')[0]})

|  | uri |
| --- | --- |
| Type | `wrcp:{name}` |
| Pydantic class | [`{pydantic}`](https://github.com/ESGF/esgf-vocab/blob/main/src/esgvoc/api/data_descriptors/{pydantic}.py): {DATA_DESCRIPTOR_CLASS_MAPPING[pydantic].__name__ if pydantic else ' Not yet implemented'} |
| | |
| JSON-LD | `{short}` |
| Content | [{io}]({io}) |
| Developer Reoo | [![Open in GitHub](https://img.shields.io/badge/Open-GitHub-blue?logo=github&style=flat-square)]({content}) |


</section>
    '''
    

    
    description = f'''

<section id="description">

## Description

</section>

'''    
    
    
    
    schema = f'''
<section id="schema">

## Content Schema

{bullets}




</section>   
'''
    

    
    usage = '''
<section id="usage">

## Usage

### Online Viewer 
To view a file in a browser use the content link with `.json` appended. eg. %(content)s/.json

### Getting a File. 

A short example of how to integrate the computed ld file into your code. 

### Framing
```js
frame = {
            "@context": "%(url)s/_context_",
            "@type": "wcrp:%(dname)s",
            "keys we want": "",
            "@explicit": True

        }
        

print(usage)

```

```python

import cmipld
cmipld.frame( %(short)s )

```
</section>

    '''%{'url': io, 'dname': dir, 'short': short, 'content': content}


    
    
    readme = f'''{info}{description}{schema}{usage}'''
    
    with open(f'{dir}README.md', 'w') as f:
        f.write(readme)
    

native-horizontal-grid-type/
------ 
 Adding native_horizontal_grid_type to DATA_DESCRIPTOR_CLASS_MAPPING
make test for url2io
source-type/
make test for url2io
native-horizontal-grid-region/
------ 
 Adding native_horizontal_grid_region to DATA_DESCRIPTOR_CLASS_MAPPING
make test for url2io
empty/
Skipping empty/ as it does not contain any JSON files.
model-calendar/
------ 
 Adding model_calendar to DATA_DESCRIPTOR_CLASS_MAPPING
make test for url2io
license/
make test for url2io
frequency/
make test for url2io


mv: rename empty/ to empty/empty/: Invalid argument


activity/
The following keys are not present in all files: ['@context', 'ui-label']
make test for url2io
native-horizontal-grid-temporal-refinement/
------ 
 Adding native_horizontal_grid_temporal_refinement to DATA_DESCRIPTOR_CLASS_MAPPING
make test for url2io
product/
make test for url2io
mip/
------ 
 Adding mip to DATA_DESCRIPTOR_CLASS_MAPPING
make test for url2io
grid-label/
make test for url2io
model-component-type/
------ 
 Adding model_component_type to DATA_DESCRIPTOR_CLASS_MAPPING
make test for url2io
resolution/
make test for url2io
native-vertical-grid-coordinate/
------ 
 Adding native_vertical_grid_coordinate to DATA_DESCRIPTOR_CLASS_MAPPING
make test for url2io
organisation/
The following keys are not present in all files: ['type', 'id', 'validation-key', 'url', 'ui-label', 'ror', 'lon', 'location', 'lat', 'labels', 'kind', 'established', 'country', 'city', 'aliases', 'acronyms', 'to', 'from', 'era', 'current', 'members', 'changes', '@context', 'status']
make test for ur

In [51]:
'universal:mip'.title()

'Universal:Mip'

In [ ]:
DATA_DESCRIPTOR_CLASS_MAPPING

In [ ]:
keys